In [1]:
import pandas as pd
from glob import glob

In [2]:
def clean_name(f):
    return f.replace('.csv','-clean.csv')

def pd_localize(df, col):
    df[col] = df.apply(lambda x: pd.Timestamp(x[col]).tz_localize('utc').tz_convert('America/Los_Angeles').strftime('%m/%d/%Y %-I:%M:%S %p'), axis=1)
    return df

In [3]:
# etherscan

glob('input/etherscan/*.csv')

infile = 'input/etherscan/export-0x81aced167afc56155c0d689d6e87210ff0101e04.csv'
outfile = clean_name(infile)

lines = open(infile,'r').readlines()
lines[0] = lines[0].replace("\n",",\n")

open(outfile,'w').writelines(lines)

cols = ['DateTime','Value_IN(ETH)','Value_OUT(ETH)','TxnFee(ETH)','Historical $Price/Eth','Method']

df = pd.read_csv(outfile,usecols=cols,dtype = {'Value_IN(ETH)':str, 'Value_OUT(ETH)':str, 'TxnFee(ETH)':str})
df = df.rename(columns={'Value_IN(ETH)':'in','Value_OUT(ETH)':'out'})

pd_localize(df,'DateTime')

df['net'] = df.apply(lambda x: '-' + x['out'] if x['out'] != '0' else x['in'],axis=1)
df = df[['DateTime','net','TxnFee(ETH)','in','out','Historical $Price/Eth','Method']]
df['TxnFee(ETH)'] = '-' + df['TxnFee(ETH)']
df.to_csv(outfile,index=False)
outfile

'input/etherscan/export-0x81aced167afc56155c0d689d6e87210ff0101e04-clean.csv'

In [4]:
# etherscan weth

glob('input/etherscan-weth/*.csv')

infile = 'input/etherscan-weth/export-token-0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2.csv'
outfile = clean_name(infile)

df = pd.read_csv(infile,dtype={'Quantity':str})
df = df.drop(columns=['Txhash','Blockno','UnixTimestamp'])

df = pd_localize(df,'DateTime')

df.to_csv(outfile,index=False)
outfile

'input/etherscan-weth/export-token-0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2-clean.csv'